In [ ]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features.build_features import obtener_distribucion_consultas

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

In [ ]:
# Carga la base de TrackCare
df_track = pd.read_csv("../data/processed/trackcare_procesada.csv")

# Deja solamente las consultas atendidas
df_track_atendidas = df_track.query("estadocita == 'Atendido'")

In [ ]:
# Obtiene resumen de consultas por anio
agrupacion_anual = ["ano", "especialidad"]
distribucion_consultas_por_anio, consultas_por_paciente_por_anio = obtener_distribucion_consultas(
    df_track_atendidas, agrupacion_anual
)

# Obtiene resumen de consultas acumuladas en el periodo
agrupacion_acumulada = ["especialidad"]
distribucion_consultas_acumuladas, consultas_por_paciente_acumuladas = (
    obtener_distribucion_consultas(df_track_atendidas, agrupacion_acumulada)
)

In [ ]:
CAMBIO_ESPECIALIDADES = {
    "ANESTESIOLOGÍA": [],
    "CARDIOLOGÍA": ["Cardiología", "Cardiología Adulto"],
    "CIRUGÍA CARDIOVASCULAR": [
        "Cardiocirugía",
        "Cirugía Adulto",
        "Cirugía General",
        "Cirugía Vascular Periférica",
    ],
    "CIRUGÍA DE TÓRAX": ["Cirugía Tórax"],
    "ENFERMEDADES RESPIRATORIAS ADULTO": ["Broncopulmonar", "Broncopulmonar Adulto"],
    "ENFERMERÍA": [],
    "EQUIPO MULTIDISCIPLINARIO": [],
    "GENÉTICA CLÍNICA": [],
    "INFECTOLOGÍA": ["Infectología Adulto"],
    "MEDICINA FÍSICA Y REHABILITACIÓN": [],
    "MEDICINA INTERNA": ["Med. Interna"],
    "NUTRICIÓN": ["Nutrición"],
    "ODONTOLOGÍA": [],
    "ONCOLOGÍA": ["Oncología"],
    "PALIATIVISTA": [],
    "PSICOLOGÍA": ["Psicología"],
    "PSIQUIATRÍA ADULTO": ["Psiquiatría Adulto"],
    "QUÍMICA Y FARMACIA": [],
    "RADIOTERAPEUTA": [],
    "TRABAJO SOCIAL": [],
}